## Import

In [1]:
import numpy as np
import regex as re
import pandas as pd
import pickle as pkl
import shapely as shp
import plotly.io as pio
import geopandas as gpd
from copy import deepcopy
from sodapy import Socrata
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Data Reading

In [2]:
opt_data_path = r"C:\Users\Marco\Documents\GitHub\commercial-activities-Milan\Data\03_Output\optimal_solutions2.pkl"
opt_data = pkl.load(open(opt_data_path, "rb"))

In [3]:
NIL_data_path = r"C:\Users\Marco\Documents\GitHub\commercial-activities-Milan\Data\02_Processed\NIL_data.pkl"
NIL_data = pkl.load(open(NIL_data_path, 'rb'))

In [4]:
food_data_path = r"C:\Users\Marco\Documents\GitHub\commercial-activities-Milan\Data\02_Processed\food_data.pkl"
food_data = pkl.load(open(food_data_path, 'rb'))

## Solution analysis

In [5]:
opt_data.solution_value.astype(float).describe()

count     87.000000
mean     106.555492
std       77.269501
min        8.310000
25%       53.290000
50%       78.200000
75%      136.878000
max      436.250000
Name: solution_value, dtype: float64

In [6]:
solution_location = []

for i, row in opt_data.iterrows():
    if row.solution_index is not None:
        solution_location.append([])
        for i in range(len(row.solution_index)):
            if row.solution_index[i] == row.building_index:
                solution_location[-1].append(row.building_location[0])
            else:
                for j in range(len(row.food_index)):
                    if row.solution_index[i] == row.food_index[j]:
                        solution_location[-1].append(row.food_shop_location[j])
    else:
        solution_location.append(None)
        
opt_data["solution_location"] = solution_location

In [7]:
fig = make_subplots(rows=2, cols=2, 
                    specs=[[{"type": "mapbox"}, {"type": "xy"}], [{"type": "xy"}, {"type": "xy"}]],
                    subplot_titles=["<b>Minimum_time_length_path</b>",
                                    "",
                                    "<b>Number_of_visited_shop</b>",
                                    "<b>Minimum_time_length</b>"])

for geom in NIL_data.geometry:
    geom = geom.exterior.coords.xy
    fig.add_trace(go.Scattermapbox(
        lat=list(geom[1]),
        lon=list(geom[0]),
        mode='lines',
        marker=dict(
            color='black',
            size=0.5,
        ),
        showlegend=False,
        hoverinfo='skip',
        fill='toself',
        fillcolor="rgba(0,0,0,0.1)"
    ),row=1, col=1)
    

for path in opt_data.solution_location:
    if path is None:
        continue
    lat = [p.y for p in path]
    lon = [p.x for p in path]
    fig.add_trace(go.Scattermapbox(
        lat=lat,
        lon=lon,
        mode='lines',
        showlegend=False,
    ))
    
    fig.add_trace(go.Scattermapbox(
        lat=lat,
        lon=lon,
        mode='markers',
        showlegend=False,
        marker=dict(
            color='red',
            size=7,
        ),
    ), row=1, col=1)
    
starting_points = opt_data.building_location.apply(lambda x: x[0])
lat = [p.y for p in starting_points]
lon = [p.x for p in starting_points]

fig.add_trace(go.Scattermapbox(
    lat=lat,
    lon=lon,
    mode='markers',
    showlegend=False,
    marker=dict(
        color='blue',
        size=7,
    ),), 
    row=1, col=1)

number_of_visited_shop = opt_data.solution_index.apply(lambda x: len(x) if x is not None else 0)

fig.add_trace(go.Histogram(x=number_of_visited_shop, 
                           histnorm='probability',
                           showlegend=False),
                     row=2, col=1)

fig.add_trace(go.Histogram(x=opt_data.solution_value.astype(float), 
                           histnorm='probability',
                           showlegend=False,
                           xbins=dict( # bins used for histogram
                           start=0,
                           end=300,
                           size=20
                        ),),
                     row=2, col=2)

fig.update_layout(title_pad_l=350,
                  mapbox=dict(
                    style="open-street-map",
                    center=dict(lat=food_data.geometry.y.mean(), lon=food_data.geometry.x.mean()),
                    zoom=10
                    ),
                  height=1200,
                  width=1200,)


fig["layout"]["xaxis2"]["title"] = "number of visited shop"
fig["layout"]["xaxis3"]["title"] = "time(min)"

fig


In [8]:
path = r"C:\Users\Marco\Documents\GitHub\commercial-activities-Milan\Data\03_Output"
pio.write_image(fig, path+"\optimal_solutions.png")